<a href="https://colab.research.google.com/github/Kealfeyne/Samsung_Bootcamp_Classic_ML/blob/main/1_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Обработаем данные

In [3]:
dataset_train = pd.read_csv("train.csv")
dataset_formula_train = pd.read_csv("formula_train.csv")
dataset_kaggletest = pd.read_csv("test.csv")
dataset_kaggleformulatest = pd.read_csv("formula_test.csv")

In [4]:
dataset_train.head(1)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence,critical_temp
0,4,88.944468,57.862692,66.361592,36.116612,1.181795,1.062396,122.90607,31.794921,51.968828,...,2.257143,2.213364,2.219783,1.368922,1.066221,1,1.085714,0.433013,0.437059,29.0


In [5]:
dataset_formula_train.head(1)

,H,He,Li,Be,B,C,N,O,F,Ne,...,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0,0,0,29.0,Ba0.2La1.8Cu1O4


In [6]:
dataset_kaggletest.head(1)

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,mean_Valence,wtd_mean_Valence,gmean_Valence,wtd_gmean_Valence,entropy_Valence,wtd_entropy_Valence,range_Valence,wtd_range_Valence,std_Valence,wtd_std_Valence
0,5,92.729214,61.051113,73.132787,37.51393,1.449309,1.146919,122.90607,35.741099,47.094633,...,2.0,2.2,1.888175,2.124829,1.557113,1.053346,2,1.114286,0.632456,0.550325


In [7]:
dataset_kaggleformulatest.head(1)

,H,He,Li,Be,B,C,N,O,F,Ne,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,material
0,0.0,0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,Ba0.1La1.9Ag0.5Cu0.5O4


In [8]:
dataset_train = dataset_train.drop(['critical_temp'], axis=1)

Что если вообще не учитывать файл с формулами или учитывать только его?


In [9]:
"""full"""
processed_dataset_train = pd.concat([dataset_train, dataset_formula_train], axis=1).drop(columns=['material'])
"""w/o formula"""
# processed_dataset_train = pd.concat([dataset_train, dataset_formula_train['critical_temp']], axis=1)
"""only formula"""
# processed_dataset_train = dataset_formula_train.drop(columns=['material'])

"""full"""
kaggletest = pd.concat([dataset_kaggletest, dataset_kaggleformulatest], axis=1).drop(columns=['material'])
"""w/o formula"""
# kaggletest = dataset_kaggletest
"""only formula"""
# kaggletest = dataset_kaggleformulatest.drop(columns=['material'])

print(processed_dataset_train.shape)
print(kaggletest.shape)

(17010, 168)
(4253, 167)


In [10]:
print(processed_dataset_train.isna().sum())

number_of_elements       0
mean_atomic_mass         0
wtd_mean_atomic_mass     0
gmean_atomic_mass        0
wtd_gmean_atomic_mass    0
                        ..
Bi                       0
Po                       0
At                       0
Rn                       0
critical_temp            0
Length: 168, dtype: int64


In [11]:
print(kaggletest.isna().sum().sort_values(ascending=False))

number_of_elements      0
Mn                      0
Co                      0
Ni                      0
Cu                      0
                       ..
range_FusionHeat        0
wtd_range_FusionHeat    0
std_FusionHeat          0
wtd_std_FusionHeat      0
Rn                      0
Length: 167, dtype: int64


Поиграемся с корреляцией

In [12]:
processed_dataset_train.corr()

,number_of_elements,mean_atomic_mass,wtd_mean_atomic_mass,gmean_atomic_mass,wtd_gmean_atomic_mass,entropy_atomic_mass,wtd_entropy_atomic_mass,range_atomic_mass,wtd_range_atomic_mass,std_atomic_mass,...,Pt,Au,Hg,Tl,Pb,Bi,Po,At,Rn,critical_temp
number_of_elements,1.000000,-0.147017,-0.359589,-0.297780,-0.460153,0.939359,0.882156,0.684415,-0.324029,0.517613,...,-0.039472,-0.027557,0.111363,0.113434,0.058097,0.245049,NaN,NaN,NaN,0.601792
mean_atomic_mass,-0.147017,1.000000,0.817031,0.941275,0.748241,-0.109166,-0.102214,0.123012,0.449137,0.193848,...,0.112087,0.056326,0.103446,0.116263,0.190016,0.123422,NaN,NaN,NaN,-0.111586
wtd_mean_atomic_mass,-0.359589,0.817031,1.000000,0.848527,0.965004,-0.315194,-0.417680,-0.145627,0.719892,-0.061539,...,0.115072,0.069147,0.041288,0.059139,0.168425,0.117003,NaN,NaN,NaN,-0.312027
gmean_atomic_mass,-0.297780,0.941275,0.848527,1.000000,0.856949,-0.196480,-0.236936,-0.175981,0.461018,-0.121441,...,0.096922,0.058219,0.034598,0.062819,0.163556,0.037827,NaN,NaN,NaN,-0.230291
wtd_gmean_atomic_mass,-0.460153,0.748241,0.965004,0.856949,1.000000,-0.377961,-0.489958,-0.350984,0.677883,-0.272322,...,0.096406,0.067551,-0.002531,0.021144,0.148240,0.032348,NaN,NaN,NaN,-0.370618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Bi,0.245049,0.123422,0.117003,0.037827,0.032348,0.163687,0.151323,0.378295,0.093136,0.339224,...,-0.032601,-0.006949,-0.048173,-0.052587,0.065463,1.000000,NaN,NaN,NaN,0.159403
Po,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
At,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
def columns_to_target_correlation(dataset, target_column='critical_temp'):
  """Возвращает корреляцию dataset c target"""
  correlation = dataset.corr()[target_column]
  correlation.columns=['Сoefficient']
  sorted_correlation = correlation.drop([target_column], axis=0).dropna().sort_values()
  return sorted_correlation

In [14]:
def get_profitable_features(correlation, restriction):
  profitable_features = correlation[(correlation > restriction) | (correlation < -restriction)].index
  return profitable_features

Попробуем учить на тех фичах, корреляция которых имеет коэффициент по модулю больше критерия

PS: не помогло :(

In [15]:
correlation = columns_to_target_correlation(processed_dataset_train)
correlation

wtd_mean_Valence              -0.632833
wtd_gmean_Valence             -0.616134
mean_Valence                  -0.601103
gmean_Valence                 -0.574112
gmean_Density                 -0.541877
                                 ...   
wtd_entropy_atomic_mass        0.626545
std_ThermalConductivity        0.652652
range_atomic_radius            0.655900
range_ThermalConductivity      0.686461
wtd_std_ThermalConductivity    0.720447
Name: critical_temp, Length: 158, dtype: float64

In [16]:
profitable_features = get_profitable_features(correlation, 0.65)
profitable_features

Index(['std_ThermalConductivity', 'range_atomic_radius',
       'range_ThermalConductivity', 'wtd_std_ThermalConductivity'],
      dtype='object')

In [17]:
correlating_features_dataset = processed_dataset_train[profitable_features]

Можно поискать еще сильно коррелирующие между собой фичи

Разделим данные

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
"""Все фичи"""
features_train, features_test, targets_train, targets_test = train_test_split(processed_dataset_train.drop(['critical_temp'], axis=1), processed_dataset_train['critical_temp'], test_size=0.3, random_state = 2)
"""Коррелирующие фичи"""
# features_train, features_test, targets_train, targets_test = train_test_split(correlating_features_dataset, processed_dataset_train['critical_temp'], test_size=0.3, random_state = 1)

print(f'Train: {features_train.shape}, {targets_train.shape}')
print(f'Test: {features_test.shape}, {targets_test.shape}')

Train: (15309, 167), (15309,)
Test: (1701, 167), (1701,)


# Рассмотрим регрессионные модели

In [20]:
from sklearn.metrics import mean_squared_error

def analyze_model(model, x_train, y_train, x_test, y_test):
  """Возвращает обученную модель, MSE"""
  model.fit(x_train, y_train)

  predictions = model.predict(x_test)
  MSE = mean_squared_error(y_test, predictions)
  return model, MSE

def get_result(model, accuracy, kaggle_features, title):
  """Скачивает csv с predictions"""
  predictions = model.predict(kaggle_features)
  result = pd.DataFrame(predictions, columns=['critical_temp']).reset_index()
  result.to_csv(f"({round(accuracy, 3)}) {title}.csv", index=False)
  files.download(f"({round(accuracy, 3)}) {title}.csv")

## Линейная регрессия

In [21]:
from sklearn.linear_model import LinearRegression
LinearRegression = LinearRegression()

In [ ]:
model_LinearRegression, MSE_LinearRegression = analyze_model(LinearRegression, features_train, targets_train, features_test, targets_test)
print(f'LinearRegression MSE : {MSE_LinearRegression}')

In [23]:
get_result(model_LinearRegression, MSE_LinearRegression, kaggletest, 'LinearRegression')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Регрессия дерева решений

In [77]:
from sklearn.tree import DecisionTreeRegressor
TreeRegressor = DecisionTreeRegressor()

In [78]:
model_TreeRegressor, MSE_TreeRegressor = analyze_model(TreeRegressor, features_train, targets_train, features_test, targets_test)
print(f'TreeRegressor MSE : {MSE_TreeRegressor}')

TreeRegressor MSE : 144.75649226121695


In [38]:
#get_result(model_TreeRegressor, MSE_TreeRegressor, kaggletest, 'TreeRegressor').shape

## Случайный лес

In [39]:
from sklearn.ensemble import RandomForestRegressor
RandomForestRegressor = RandomForestRegressor()

In [108]:
model_RandomForestRegressor, MSE_RandomForestRegressor = analyze_model(RandomForestRegressor, features_train, targets_train, features_test, targets_test)
print(f'RandomForestRegressor MSE : {MSE_RandomForestRegressor}')

RandomForestRegressor MSE : 77.16079810802542


In [109]:
get_result(model_RandomForestRegressor, MSE_RandomForestRegressor, kaggletest, 'TreeRegressor')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Градиентный спуск в регрессии

In [119]:
from sklearn.ensemble import GradientBoostingRegressor
GradientBoostingRegressor = GradientBoostingRegressor()

In [120]:
model_GradientBoostingRegressor, MSE_GradientBoostingRegressor = analyze_model(GradientBoostingRegressor, features_train, targets_train, features_test, targets_test)
print(f'GradientBoostingRegressor MSE : {MSE_GradientBoostingRegressor}')

GradientBoostingRegressor MSE : 158.81143382211246


## Хист-градиентный спуск в регрессии

In [121]:
from sklearn.ensemble import HistGradientBoostingRegressor
HistGradientBoostingRegressor = HistGradientBoostingRegressor()

In [122]:
model_HistGradientBoostingRegressor, MSE_HistGradientBoostingRegressor = analyze_model(HistGradientBoostingRegressor, features_train, targets_train, features_test, targets_test)
print(f'HistGradientBoostingRegressor MSE : {MSE_HistGradientBoostingRegressor}')

HistGradientBoostingRegressor MSE : 104.52802743679058


## Lasso

In [123]:
from sklearn.linear_model import LassoCV
LassoCV = LassoCV()

In [124]:
model_LassoCV, MSE_LassoCV = analyze_model(LassoCV, features_train, targets_train, features_test, targets_test)
print(f'LassoCV MSE : {MSE_LassoCV}')

LassoCV MSE : 285.8459322084308


## Нейросеть

In [125]:
from sklearn.neural_network import MLPRegressor
from tqdm import tqdm

In [ ]:
best_MSE_1 = 0
best_params_1 = 0
for i in tqdm(range(1, 50)):
  _MLPRegressor = MLPRegressor(hidden_layer_sizes=(i), activation="tanh", max_iter=200)
  model_MLPRegressor, MSE_MLPRegressor = analyze_model(_MLPRegressor, features_train, targets_train,features_test, targets_test)
  if i == 1:
    best_MSE_1
  if MSE_MLPRegressor < best_MSE_1:
    best_MSE = MSE_MLPRegressor
    best_params_1 = i

In [ ]:
print(best_MSE_1)
print(best_params_1)

In [ ]:
#264 - [18, 1]

In [ ]:
best_MSE_2 = 300
best_params_2 = []
for i in tqdm(range(1, 20)):
  for j in range(1, 20):
    _MLPRegressor = MLPRegressor(hidden_layer_sizes=(i, j), activation="relu")
    model_MLPRegressor, MSE_MLPRegressor = analyze_model(_MLPRegressor, features_train, targets_train,features_test, targets_test)
    if MSE_MLPRegressor < best_MSE_2:
      best_MSE_2 = MSE_MLPRegressor
      best_params_2 = [i, j]

In [ ]:
#223 - [11, 18]